# Losses subpackage

The `opr.losses` subpackage contains ready-to-use loss functions implemented in PyTorch, featuring a common interface.

## Usage example - Batch Hard Triplet Margin Loss

In [1]:
import torch

from opr.models.place_recognition import MinkLoc3D
from opr.datasets import OxfordDataset
from opr.samplers import BatchSampler
from opr.losses import BatchHardTripletMarginLoss


In [2]:
dataset = OxfordDataset(
    dataset_root="/home/docker_opr/Datasets/pnvlad_oxford_robotcar_full",
    subset="train",
    data_to_load=["pointcloud_lidar"],
)

sampler = BatchSampler(
    dataset=dataset,
    batch_size=32,
    positives_per_group=4
)

dataloader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_sampler=sampler,
    num_workers=4,
    pin_memory=True,
    collate_fn=dataset.collate_fn,
)


In [3]:
sample_batch = next(iter(dataloader))
sample_batch = {k: v.to("cuda") for k, v in sample_batch.items()}
sample_batch.keys()


dict_keys(['idxs', 'utms', 'pointclouds_lidar_coords', 'pointclouds_lidar_feats'])

In [4]:
model = MinkLoc3D()
model = model.to("cuda")
model.train();


In [5]:
output = model(sample_batch)
output.keys()


dict_keys(['final_descriptor'])

In [6]:
loss_fn = BatchHardTripletMarginLoss(margin=0.2)


In [7]:
idxs = sample_batch["idxs"]
positives_mask = dataset.positives_mask[idxs][:, idxs]
negatives_mask = dataset.negatives_mask[idxs][:, idxs]

loss, stats = loss_fn(output["final_descriptor"], positives_mask, negatives_mask)


In [8]:
loss


tensor(4.2445, device='cuda:0', grad_fn=<MeanBackward0>)

In [9]:
stats


{'loss': 4.244545936584473,
 'avg_embedding_norm': 22.514673233032227,
 'num_triplets': 32,
 'num_non_zero_triplets': 32.0,
 'non_zero_rate': 1.0,
 'max_pos_pair_dist': 12.201509475708008,
 'max_neg_pair_dist': 7.6710076332092285,
 'mean_pos_pair_dist': 6.20413875579834,
 'mean_neg_pair_dist': 2.3874855041503906,
 'min_pos_pair_dist': 1.6780879497528076,
 'min_neg_pair_dist': 1.2022242546081543}